# Project 2: ISS Pass Predictor & Logger

---

**Project Overview**
--

This project creates a personalized International Space Station (ISS) visibility tracker that predicts when the station passes overhead and maintains a historical log of observations. We'll use Skyfield for satellite predictions while building a Pandas-based database to track and analyze pass patterns over time.

**Key Features:**
- Download ISS TLE (Two-Line Element) data from Celestrak
- Predict visible passes for the next 7 days using Skyfield
- Store predictions in structured DataFrames
- Log actual observations with weather and viewing success data
- Analyze patterns using Pandas groupby operations
- Visualize pass frequencies and observation success rates
- Persist data to CSV files for cross-session continuity

**Astronomy Context:**
The International Space Station orbits Earth every ~90 minutes. A "pass" occurs when it flies overhead and becomes visible - appearing as a bright moving star. TLE data contains orbital parameters that enable software to predict satellite positions. Altitude refers to height above the horizon in degrees (0° = horizon, 90° = directly overhead). Passes above ~30° altitude are easiest to observe.

---

### Methodology

**Pathway B: Pandas + Astropy + Skyfield**
- **Skyfield**: Satellite orbit calculations and pass predictions
- **Pandas**: Data manipulation, filtering, and analysis
- **Astropy**: Astronomical coordinate transformations and constellation identification
- **Matplotlib**: Data visualization and plotting

**Data Pipeline:**
1. Acquire current ISS TLE data from Celestrak
2. Calculate visible passes for user location
3. Structure predictions in DataFrame format
4. Log observations with metadata
5. Merge predictions with observations
6. Analyze patterns and generate visualizations
7. Persist data for future sessions

## 1. Project Setup, Dependencies, and Data Acquisition

This section establishes the foundation for our ISS tracking system by:
- Installing required dependencies
- Importing necessary libraries
- Configuring the observing location
- Acquiring current ISS orbital data
- Setting up data structures for predictions and observations

Let's begin by ensuring all required packages are available and properly imported.

### 1.1 Dependency Installation

**⚠️ IMPORTANT: Run this cell first before running any import cells!**

Before we can start predicting ISS passes, we need to install the required Python libraries. Each library serves a specific purpose in our project:

- **pandas**: Core data manipulation and analysis tool. We'll use this to store predictions in DataFrames, filter passes, and perform groupby operations.
- **numpy**: Numerical computing foundation that pandas is built upon.
- **matplotlib**: Visualization library for creating charts and plots of pass frequencies.
- **skyfield**: Specialized library for satellite orbit calculations and predicting when satellites are visible.
- **astropy**: Astronomical coordinate transformations for identifying constellations and working with celestial coordinates.
- **requests**: HTTP library for downloading TLE data from Celestrak website.

**Important Notes:**
- Run the installation cell below before proceeding to imports
- The installation may take 1-2 minutes depending on your internet connection
- Skyfield will automatically download ephemeris data (~10MB) on first use - this is normal
- If you get import errors, make sure you've run the installation cell and restarted the kernel

In [3]:
# Install all required packages
# Run this cell FIRST before any import statements!
# This command downloads and installs the libraries if they're not already present
%pip install pandas numpy matplotlib skyfield astropy requests

Note: you may need to restart the kernel to use updated packages.


### 1.2 Library Imports

**⚠️ Make sure you've run the installation cell above (Cell 3) before running this cell!**

Now that we have installed the packages, we need to import them into our notebook. Importing brings the library functions and classes into our current namespace so we can use them.

**If you get a `ModuleNotFoundError`:**
1. Go back and run Cell 3 (the installation cell)
2. Wait for installation to complete
3. Restart the kernel (Kernel → Restart)
4. Run Cell 3 again, then run this cell

**What we're importing:**
- Standard data science libraries (pandas, numpy, matplotlib)
- Date/time utilities for handling timestamps
- Skyfield classes for satellite tracking
- Astropy classes for astronomical coordinates


In [4]:
# Import all necessary libraries

# Standard Python libraries for data manipulation and web requests
import pandas as pd              # Primary tool for DataFrames and data analysis
import numpy as np               # Numerical computing (pandas dependency)
import requests                  # For downloading TLE data from Celestrak
import matplotlib.pyplot as plt  # For creating visualizations
from datetime import datetime, timedelta  # For handling dates and times
import warnings                  # To suppress non-critical warnings
warnings.filterwarnings('ignore')

# Skyfield imports - for satellite orbit calculations
from skyfield.api import load, EarthSatellite, Topos
# load: downloads ephemeris data
# EarthSatellite: represents a satellite with TLE data
# Topos: represents an observer location on Earth

# Astropy imports - for astronomical coordinate transformations
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_constellation
from astropy import units as u   # Physical units (degrees, meters, etc.)
from astropy.time import Time    # Time handling for astronomical calculations

print("All libraries imported successfully!")


✓ All libraries imported successfully!


### 1.3 Define Observing Location

To predict when the ISS will be visible, we need to know our location on Earth. The ISS pass predictions depend on where we're observing from - different locations see the ISS at different times.

For this project, we'll use **Columbus, Ohio** as our default location. You can easily change the coordinates below to your own location.

**What we need:**
- **Latitude**: Distance north or south of equator (positive = north, negative = south)
- **Longitude**: Distance east or west of prime meridian (positive = east, negative = west)  
- **Elevation**: Height above sea level in meters (affects horizon calculations)

**Why this matters**: The ISS orbits at ~400km altitude, but passes appear different depending on our location. A pass visible from Ohio might not be visible from California!

In [ ]:
# Step 1: Define geographic coordinates
# Columbus, Ohio coordinates
latitude = 39.9612    # North latitude in degrees (positive = north of equator)
longitude = -82.9988  # West longitude in degrees (negative = western hemisphere)
elevation = 275       # Elevation above sea level in meters

print("Observing Location Configuration:")
print(f"  Latitude: {latitude}° N")
print(f"  Longitude: {abs(longitude)}° W")  # Display as positive for clarity
print(f"  Elevation: {elevation} m above sea level")
print()

# Step 2: Create Skyfield observer object
# Topos() creates a location object that Skyfield uses for calculations
observer = Topos(latitude, longitude, elevation_m=elevation)
print("Observer location object created")

Observing Location Configuration:
  Latitude: 39.9612° N
  Longitude: 82.9988° W
  Elevation: 275 m above sea level

✓ Observer location object created


### 1.4 Load Skyfield Timescale and Ephemeris Data

Skyfield needs two types of data files to make accurate predictions:

1. **Timescale**: A system for handling dates and times in astronomical calculations. It accounts for things like leap seconds and different time systems (UTC, etc.)

2. **Ephemeris Data**: A table of precise positions for celestial bodies (Earth, Sun, Moon, planets). This allows Skyfield to calculate exactly where satellites will appear in the sky.

**Note**: On first run, Skyfield will automatically download the ephemeris file (~10MB). This only happens once - subsequent runs use the cached file.


In [ ]:
# Step 1: Load Skyfield timescale
# The timescale object handles time conversions for astronomical calculations
print("Loading Skyfield timescale...")
ts = load.timescale()
print("✓ Timescale loaded successfully")
print()

# Step 2: Load ephemeris data
# This downloads JPL DE421 ephemeris (~10MB) on first run
# The ephemeris contains precise positions of Earth, Sun, Moon, and planets
print("Loading ephemeris data (this may download ~10MB on first run)...")
eph = load('de421.bsp')
print("Ephemeris data loaded successfully")

print("\n" + "="*50)
print("Location and astronomical data setup complete!")
print("="*50)


Loading Skyfield timescale...
✓ Timescale loaded successfully

Loading ephemeris data (this may download ~10MB on first run)...


[#################################] 100% de421.bsp


✓ Ephemeris data loaded successfully

Location and astronomical data setup complete!


### 1.5 Acquire TLE Data for the ISS

**What is TLE Data?**

TLE (Two-Line Element) data is a standardized format that contains all the information needed to predict a satellite's position. Think of it as a "snapshot" of the satellite's orbit at a specific moment in time.

**Key facts about TLE data:**
- **Format**: Three lines - satellite name, then two lines of orbital parameters
- **Updated regularly**: ISS TLE data is updated daily because orbits decay slightly over time
- **Contains**: Orbital inclination, altitude, speed, and other parameters
- **ISS NORAD ID**: 25544 (a unique identifier assigned by the North American Aerospace Defense Command)

**Why we need it**: Without current TLE data, our predictions would be inaccurate. The ISS orbit changes slightly each day due to atmospheric drag, so we need fresh data from Celestrak (a trusted source for satellite tracking data).

In [ ]:
# Step 1: Set up the URL for downloading TLE data
# Celestrak provides free, regularly-updated TLE data for all satellites
tle_url = "https://celestrak.org/NORAD/elements/stations.txt"
print("Downloading ISS TLE data from Celestrak...")
print(f"Source: {tle_url}")
print()

# Step 2: Download the TLE data file
try:
    # Make HTTP request to download the file
    response = requests.get(tle_url)
    # Check if download was successful (status code 200 = success)
    response.raise_for_status()
    
    # Step 3: Parse the downloaded text
    # TLE files are plain text with each satellite's data on 3 lines
    tle_lines = response.text.strip().split('\n')
    print(f"Downloaded {len(tle_lines)} lines of TLE data")
    
    # Step 4: Find ISS data in the file
    # We search for lines containing "ISS" or the NORAD ID "25544"
    iss_tle = None
    for i, line in enumerate(tle_lines):
        if 'ISS' in line or '25544' in line:
            # ISS TLE format: name (line i), TLE line 1 (line i+1), TLE line 2 (line i+2)
            if i + 2 < len(tle_lines):
                iss_tle = tle_lines[i:i+3]
                break
    
    # Step 5: Process the ISS TLE data
    if iss_tle:
        print("\n✓ ISS TLE data found and downloaded successfully")
        print(f"  Satellite name: {iss_tle[0].strip()}")
        print(f"  TLE line 1: {iss_tle[1].strip()[:70]}...")  # Show first part
        print(f"  TLE line 2: {iss_tle[2].strip()[:70]}...")
        
        # Create a Skyfield EarthSatellite object from the TLE data
        # This object will be used for all our pass calculations
        satellite = EarthSatellite(iss_tle[1], iss_tle[2], iss_tle[0], ts)
        print("\nISS satellite object created - ready for calculations!")
        
    else:
        # If ISS not found, something went wrong
        print("\n✗ Error: ISS TLE data not found in downloaded file")
        raise ValueError("ISS data not found")
        
except (requests.exceptions.RequestException, ValueError) as e:
    # Step 6: Handle errors gracefully with fallback data
    print(f"\n✗ Error downloading/parsing TLE data: {e}")
    print("Using fallback TLE data (may be outdated, but allows testing)...")
    
    # Fallback TLE data structure (placeholder values)
    # In real use, this allows the notebook to run even if network fails
    fallback_tle = [
        "ISS (ZARYA)",
        "1 25544U 98067A   24123.12345678  .00001234  00000-0  12345-4 0  9999",
        "2 25544  51.6400 123.4567 0001234 123.4567 234.5678 15.12345678901234"
    ]
    
    satellite = EarthSatellite(fallback_tle[1], fallback_tle[2], fallback_tle[0], ts)
    print("Fallback satellite object created (for testing purposes)")

print("\n" + "="*50)
print("TLE data acquisition complete!")
print("="*50)

Source: https://celestrak.org/NORAD/elements/stations.txt

Downloaded 72 lines of TLE data

✓ ISS TLE data found and downloaded successfully
  Satellite name: ISS (ZARYA)
  TLE line 1: 1 25544U 98067A   25304.16626396  .00017194  00000+0  31236-3 0  9992...
  TLE line 2: 2 25544  51.6343 353.2496 0004910 359.9780   0.1208 15.49645314536255...

✓ ISS satellite object created - ready for calculations!

TLE data acquisition complete!


## 2. Core Prediction Pipeline and Data Wrangling

This section implements the Skyfield pass calculation and structures the output into a Pandas DataFrame. We'll:

1. Calculate all visible ISS passes for the next 7 days
2. Extract key information: rise time, max altitude, duration, brightness
3. Store everything in a DataFrame for easy analysis
4. Filter passes by altitude threshold (passes above 30° are easiest to see)

**Key Concepts:**
- A "pass" is when the ISS rises above the horizon, reaches maximum altitude, and sets
- We'll calculate passes where the ISS reaches at least 10° altitude (visible above the horizon)
- Duration is how long the pass lasts from rise to set
- Max altitude tells us how high the ISS appears in the sky

### 2.1 Function to Calculate Visible Passes

This is the core function that predicts when the ISS will be visible. It uses Skyfield's `find_events()` method to detect when the satellite rises above the horizon, reaches its highest point, and sets below the horizon.

**How it works:**
1. Skyfield calculates the ISS position at many time points
2. It detects when altitude crosses our minimum threshold (e.g., 10°)
3. For each complete pass, it records:
   - **Rise time**: When ISS first appears above horizon
   - **Max altitude time**: When ISS reaches highest point
   - **Set time**: When ISS disappears below horizon
   - **Max altitude**: How high in degrees (0° = horizon, 90° = overhead)
   - **Duration**: Total time from rise to set in minutes
   - **Brightness**: Apparent magnitude (lower number = brighter star)

**What we extract for each pass:**
- Rise time → when to start watching
- Max altitude → how high ISS will appear (higher = easier to see)
- Duration → how long we have to observe
- Brightness → how bright it will appear (depends on distance and solar angle)

The function returns a list of dictionaries, where each dictionary contains information about one pass.

In [8]:
def calculate_visible_passes(satellite, observer_location, start_time, days=7, min_altitude=10.0):
    """
    Calculate all visible ISS passes for a specified time period.
    
    This function uses Skyfield to find when the ISS rises above the horizon,
    reaches its highest point (culmination), and sets below the horizon. For
    each complete pass, it extracts key information like timing, altitude, and brightness.
    
    Parameters:
    -----------
    satellite : skyfield.api.EarthSatellite
        The ISS satellite object created from TLE data
    observer_location : skyfield.toposlib.Topos
        The observer's location on Earth (created from latitude/longitude)
    start_time : skyfield.timelib.Time
        When to start searching for passes (usually current time)
    days : float, optional
        How many days ahead to calculate passes (default: 7)
    min_altitude : float, optional
        Minimum altitude in degrees above horizon to consider (default: 10.0)
        Passes below this won't be included (helps filter out very low passes)
    
    Returns:
    --------
    list of dict
        Each dictionary contains:
        - 'rise_time': datetime when ISS appears above horizon
        - 'max_alt_time': datetime when ISS reaches highest point
        - 'set_time': datetime when ISS disappears below horizon
        - 'max_altitude': float, highest altitude in degrees (0-90)
        - 'duration_minutes': float, total pass duration
        - 'brightness': float, apparent magnitude (lower = brighter)
    
    Example:
    --------
    >>> passes = calculate_visible_passes(satellite, observer, ts.now(), days=7)
    >>> print(f"Found {len(passes)} passes")
    """
    
    # Step 1: Calculate the end time for our search window
    end_time = start_time + days
    
    # Step 2: Use Skyfield to find all rise/culminate/set events
    # find_events() returns:
    #   - t: array of times when events occur
    #   - events: array of event types (0=rise, 1=culminate, 2=set)
    # Events are only recorded when altitude crosses min_altitude threshold
    t, events = satellite.find_events(observer_location, start_time, end_time, altitude_degrees=min_altitude)
    
    # Step 3: Initialize list to store pass information
    passes = []
    
    # Step 4: Loop through events and group them into complete passes
    # Each complete pass has 3 events in order: rise (0), culminate (1), set (2)
    i = 0
    while i < len(events):
        # Check if we have a complete pass sequence (rise -> culminate -> set)
        if i + 2 < len(events) and events[i] == 0 and events[i+1] == 1 and events[i+2] == 2:
            
            # Extract the three key times for this pass
            rise_time = t[i]        # When ISS first appears
            max_alt_time = t[i+1]    # When ISS is highest
            set_time = t[i+2]       # When ISS disappears
            
            # Step 5: Calculate pass duration
            # Skyfield times are in days, so convert to minutes
            duration_minutes = (set_time - rise_time) * 24 * 60
            
            # Step 6: Calculate maximum altitude (how high ISS appears)
            # We need the satellite's position relative to our location
            difference = satellite - observer_location  # Relative position vector
            topocentric = difference.at(max_alt_time)    # Position at culmination time
            alt, az, distance = topocentric.altaz()     # Get altitude, azimuth, distance
            max_altitude = alt.degrees                   # Convert to degrees
            
            # Step 7: Estimate brightness (apparent magnitude)
            # Brightness depends on: distance to ISS, solar phase angle, ISS size
            # This is a simplified approximation - real calculations are more complex
            # ISS magnitude typically ranges from -2 (very bright) to +3 (dim)
            distance_km = distance.km
            brightness = 0.0 - (400 / distance_km)  # Closer ISS = brighter (lower number)
            
            # Step 8: Store all information about this pass in a dictionary
            pass_info = {
                'rise_time': rise_time.utc_datetime(),      # Convert to Python datetime
                'max_alt_time': max_alt_time.utc_datetime(),
                'set_time': set_time.utc_datetime(),
                'max_altitude': round(max_altitude, 2),     # Round to 2 decimal places
                'duration_minutes': round(duration_minutes, 1),  # Round to 1 decimal
                'brightness': round(brightness, 2)         # Round to 2 decimal places
            }
            
            passes.append(pass_info)
            i += 3  # Move to next potential pass (skip the 3 events we just processed)
        else:
            i += 1  # Skip incomplete events (if rise/culminate/set don't match)
    
    return passes

# Set up time period: next 7 days from now
now = ts.now()
print(f"Calculating passes starting from: {now.utc_datetime()}")
print(f"Looking ahead: 7 days")
print("="*50)

# Calculate all visible passes
all_passes = calculate_visible_passes(satellite, observer, now, days=7)

print(f"Found {len(all_passes)} visible passes in the next 7 days!")
print("="*50)


Calculating passes starting from: 2025-10-31 19:03:34.228681+00:00
Looking ahead: 7 days
Found 42 visible passes in the next 7 days!


Now let's call our function to calculate all visible passes for the next 7 days. We'll start from the current time and look ahead 7 days.


### 2.2 Store Predictions in DataFrame

Now we'll convert our list of passes into a Pandas DataFrame. This makes it easy to:
- View all passes in a table
- Filter passes by altitude
- Analyze patterns
- Save to CSV for later use

We'll create a DataFrame with all the pass information, then filter for passes above 30° altitude (easiest to see).


In [9]:
# Convert list of passes to a DataFrame
predictions_df = pd.DataFrame(all_passes)

# Display basic info about our DataFrame
print("Pass Predictions DataFrame:")
print(f"Total passes found: {len(predictions_df)}")
print("\nFirst few passes:")
print(predictions_df.head())

print("\n" + "="*50)
print("DataFrame Info:")
print(predictions_df.info())
print("="*50)


Pass Predictions DataFrame:
Total passes found: 42

First few passes:
                         rise_time                     max_alt_time  \
0 2025-11-01 03:21:54.374539+00:00 2025-11-01 03:23:28.194187+00:00   
1 2025-11-01 04:56:31.125321+00:00 2025-11-01 04:59:50.166486+00:00   
2 2025-11-01 06:34:54.443733+00:00 2025-11-01 06:37:06.348067+00:00   
3 2025-11-01 08:13:44.427059+00:00 2025-11-01 08:14:58.320281+00:00   
4 2025-11-01 09:49:57.000536+00:00 2025-11-01 09:52:41.920422+00:00   

                          set_time  max_altitude  duration_minutes  brightness  
0 2025-11-01 03:25:02.578869+00:00         12.78               3.1       -0.30  
1 2025-11-01 05:03:10.627237+00:00         67.26               6.7       -0.89  
2 2025-11-01 06:39:18.711783+00:00         16.16               4.4       -0.34  
3 2025-11-01 08:16:12.132916+00:00         11.51               2.5       -0.28  
4 2025-11-01 09:55:26.712004+00:00         22.63               5.5       -0.42  

DataFrame Info:


### 2.3 Filter Passes by Altitude Threshold

Passes above 30° altitude are easiest to see because they're high in the sky and above most obstructions. We'll filter our DataFrame to show only these "good" passes.

In [10]:
# Filter passes by altitude threshold (30 degrees = easiest to see)
altitude_threshold = 30.0

# Use Pandas to filter: only keep passes where max_altitude >= threshold
good_passes_df = predictions_df[predictions_df['max_altitude'] >= altitude_threshold]

print(f"Filtering passes above {altitude_threshold}° altitude...")
print(f"Total passes: {len(predictions_df)}")
print(f"Good passes (above {altitude_threshold}°): {len(good_passes_df)}")
print("\nBest viewing opportunities:")
print(good_passes_df[['rise_time', 'max_altitude', 'duration_minutes', 'brightness']].head(10))

print("\n" + "="*50)
print("Filtering complete!")
print("="*50)

Filtering passes above 30.0° altitude...
Total passes: 42
Good passes (above 30.0°): 15

Best viewing opportunities:
                          rise_time  max_altitude  duration_minutes  \
1  2025-11-01 04:56:31.125321+00:00         67.26               6.7   
5  2025-11-01 11:26:21.825928+00:00         56.49               6.6   
6  2025-11-02 04:08:26.791338+00:00         68.85               6.6   
10 2025-11-02 10:38:13.637168+00:00         80.55               6.7   
12 2025-11-03 03:20:35.961606+00:00         36.99               6.2   
16 2025-11-03 09:50:09.729455+00:00         46.96               6.5   
19 2025-11-04 04:09:03.093012+00:00         41.86               6.4   
22 2025-11-04 09:02:08.016620+00:00         30.51               6.1   
23 2025-11-04 10:38:52.260173+00:00         32.61               6.1   
25 2025-11-05 03:20:41.142427+00:00         70.87               6.7   

    brightness  
1        -0.89  
5        -0.80  
6        -0.90  
10       -0.94  
12       -0.61  

**Observation Log and Merge**

We define the structure of the observation log (observations.csv), create a dummy dataset, and merge it with the predictions.

**Analysis, Visualization, and Conclusion**
--
This section fulfills the analysis, visualization, and persistence requirements.

---

**Analysis: Pass Frequency per Day**

We analyze the frequency of passes over the next 7 days, which is a key requirement.

**Visualization: Pass Frequencies (Bar Chart)**

A bar chart visualizing the number of passes per day.

**Visualization: Observation Success Rate**

A pie chart showing the success rate of the recorded observations.

**Persistence: Save Predictions and Full Data**

Save the full dataset to a new CSV file to meet the persistence requirement.

**Conclusion and Individual Contribution**
--

---

**Discussion and Interpretation**

**Individual Contribution Statement**